In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [2]:
# , na_values='[]')
df_movies = pd.read_csv('../../datasets/tmdb_5000_movies.csv')
df_credit = pd.read_csv('../../datasets/tmdb_5000_credits.csv')
df_credit.drop('title', axis=1, inplace=True)
df_all = pd.merge(df_movies, df_credit, left_on='id', right_on='movie_id')
df_all = df_all[df_movies.status == 'Released']
df_all.dropna(subset=['overview', 'genres', 'keywords'], inplace=True, )
df_all[['overview', 'genres', 'keywords']].fillna(' ', inplace=True)
print('Количество фильмов после фильтрации:', df_all.shape[0])

Количество фильмов после фильтрации: 4792


## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import ast


df_all['combined'] = df_all.keywords.apply(
    lambda x: " ".join([str['name'] for str in ast.literal_eval(x)]))
df_all['combined'] = df_all.combined + ' ' + df_all.overview

vec = TfidfVectorizer(stop_words='english', max_features=10000)
# df_all['overview'] = df_all['overview'].fillna('')
matrix = vec.fit_transform(df_all.combined)
matrix.shape

(4792, 10000)

## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [15]:
from sklearn.metrics.pairwise import cosine_similarity


cosine_sim = cosine_similarity(matrix)
df_distance = pd.DataFrame(cosine_sim, index=df_all.id, columns=df_all.id)

In [9]:
df_distance.to_csv('../assets/distance.csv')
df_all.to_csv('../assets/movies.csv')

In [16]:
df_distance.head()

id,19995,285,206647,49026,49529,559,38757,99861,767,209112,...,182291,286939,124606,14337,67238,9367,72766,231617,126186,25975
id,,,,,,,,,,,,,,,,,,,,,
19995,1.000000,0.006698,0.000000,0.011519,0.193628,0.043216,0.008125,0.048032,0.000000,0.012418,...,0.000000,0.0,0.017703,0.033747,0.000000,0.000000,0.0,0.004940,0.013178,0.00000
285,0.006698,1.000000,0.000000,0.014123,0.040554,0.023146,0.004609,0.014998,0.000000,0.000000,...,0.000000,0.0,0.014527,0.000000,0.000000,0.009755,0.0,0.019802,0.000000,0.00000
206647,0.000000,0.000000,1.000000,0.018515,0.014561,0.014341,0.019677,0.047230,0.016920,0.006142,...,0.031525,0.0,0.000000,0.000000,0.017365,0.014662,0.0,0.011729,0.000000,0.00000
49026,0.011519,0.014123,0.018515,1.000000,0.006272,0.024840,0.024703,0.040374,0.011192,0.226748,...,0.006161,0.0,0.004955,0.018283,0.010281,0.000000,0.0,0.028046,0.035781,0.01766
49529,0.193628,0.040554,0.014561,0.006272,1.000000,0.020283,0.020632,0.056310,0.000000,0.014569,...,0.005251,0.0,0.000000,0.010926,0.000000,0.000000,0.0,0.004216,0.000000,0.00000


In [17]:
df_all.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew,combined
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",culture clash future space war space colony so...
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",ocean drug abuse exotic island east india trad...
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",spy based on novel secret agent sequel mi6 bri...
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",dc comics crime fighter terrorist secret ident...
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",based on novel mars medallion space travel pri...
